<div class='bar_title'></div>

*Simulation for Decision Making (S4DM)*

# Introduction to Simulation with Python (Part 1)

Gunther Gust & Ignacio Ubeda <br>
Chair for Enterprise AI <br>
Data Driven Decisions Group <br>
Center for Artificial Intelligence and Data Science (CAIDAS)


<img src="images/d3.png" style="width:20%; float:left;" />

<img src="images/CAIDASlogo.png" style="width:20%; float:left;" />

# Agenda

* Introduction to Simpy
* Simpy basics
* Process interactions
* Shared resources

Credits: The following content is adapted from the official [Simpy documentation](https://simpy.readthedocs.io/en/latest/simpy_intro/index.html) 

In [1]:
import simpy

## Basic Concepts

SimPy is a discrete-event simulation library for prorgamming simulation models with Python. 


The behavior of active components (like vehicles, customers or messages) is modeled with __processes.__ All processes live in an __environment.__ They interact with the environment and with each other via __events.__

**Our First Process**

Our first example will be a process for a car. The car will alternately drive and park for a while. When it starts driving (or parking), it will print the current simulation time.

In [28]:
parking_duration = 5
trip_duration = 2

def park_and_drive(env):
    while True:
        print('Car starts parking at %d' % env.now)
        yield env.timeout(parking_duration)

        print('Car starts driving at %d' % env.now)
        yield env.timeout(trip_duration)

Processes are described by Python __generators__: 

*  __Conceptually:__ Imagine you're in a large library, looking for books on a very specific topic. Instead of bringing you the entire section at once, which could be overwhelming and unnecessary, a helpful librarian brings you one book at a time. You can look through each book, decide if it's what you need, and only when you're ready, ask for the next one. This way, you don't have to deal with more books than you can handle at one time, and the library doesn't need to allocate space and resources to deliver all the books to you at once. 
In programming, especially with Python, a generator works somewhat like this helpful librarian. A generator is a special kind of function that, instead of returning all the results at once (like bringing the whole section of books to you), yields one result at a time, pausing in between each until the next result is requested.


* __Syntactically:__ The syntax of a Python generator is quite straightforward and elegant, once you get the hang of it. To define a generator, you typically use a function just like you would for any other operation in Python. However, the key difference lies in how the function yields its result. Instead of returning a value with the `return` keyword and ending the function, a generator function uses the `yield` keyword to provide a value to the caller and then pauses its execution, maintaining its state for the next time it's called.

In simpy, you can call them __process function__ (or process method, depending on whether it is a normal function or method of a class). During their lifetime, they 

* __create events__ 
* and __yield them__
* and then __wait__ for them to be triggered.

When a process __yields an event,__ the process gets suspended. SimPy resumes the process, when the event occurs (we say that the event is triggered). Multiple processes can wait for the same event. SimPy resumes them in the same order in which they yielded that event.


An important event type is the __Timeout.__ Events of this type are triggered after a certain amount of (simulated) time has passed. They allow a process to sleep (or hold its state) for the given time. A Timeout and all other events can be created by calling the appropriate method of the Environment that the process lives in (`Environment.timeout()` for example).

Our car process requires a reference to an Environment (env) in order to create new events. The car’s behavior is described in an infinite loop. Remember, this function is a generator. Though it will never terminate, it will pass the control flow back to the simulation once a yield statement is reached. Once the yielded event is triggered (“it occurs”), the simulation will resume the function at this statement.

As I said before, our car switches between the states parking and driving. It announces its new state by printing a message and the current simulation time (as returned by the Environment.now property). It then calls the Environment.timeout() factory function to create a Timeout event. This event describes the point in time the car is done parking (or driving, respectively). By yielding the event, it signals the simulation that it wants to wait for the event to occur.


Now that the behavior of our car has been modeled, lets create an instance of it and see how it behaves:

In [29]:
env = simpy.Environment() #create environment
env.process(park_and_drive(env)) #add the "park_and_drive" process to the environment

<Process(park_and_drive) object at 0x25bfdd8e5e0>

The first thing we need to do is to create an instance of Environment. This instance is passed into our car process function. Calling it creates a process generator that needs to be started and added to the environment via Environment.process(). Note, that at this time, none of the code of our process function is being executed. Its execution is merely scheduled at the current simulation time.

The Process returned by process() can be used for process interactions (we will cover that in the next section, so we will ignore it for now).

Finally, we start the simulation by calling run() and passing an end time to it:

In [30]:
env.run(until=30)

Car starts parking at 0
Car starts driving at 5
Car starts parking at 7
Car starts driving at 12
Car starts parking at 14
Car starts driving at 19
Car starts parking at 21
Car starts driving at 26
Car starts parking at 28


__Control questions (now it's your turn):__
* Alter the timespan of the simulation to 60 timesteps and run it.
* Alter the parking duration of the car to 10 timesteps and the driving duration to 1 timestep.
* Run a simulation with two (identical) cars.
* Create a second process named "truck_park_and_drive". Since a truck is slower, both its driving and parking durations should be larger than the ones of the car. Then, run a simulation that contains both a truck and a car.  

## Process Interaction

The Process instance that is returned by Environment.process() can be utilized for process interactions. The two most common examples for this are to **wait for another process to finish** and to **interrupt another process** while it is waiting for an event.

**Waiting for a Process**

As it happens, a SimPy Process can be used like an event (technically, a process actually is an event). If you yield a process (behind the `yield` statement), you are resumed once the process has finished. 
* Imagine a car-wash simulation where cars enter the car-wash and wait for the washing process to finish. 
* Or an airport simulation where passengers have to wait until a security check finishes.


Lets assume that the car from our last example magically became an electric vehicle. Electric vehicles usually take a lot of time charging their batteries after a trip. They have to wait until their battery is charged before they can start driving again.


We can model this with an additional charge() process for our car. Therefore, we refactor our car to be a class with two process methods: `park_and_drive()` (which is the original process function) and `charge()`:


In [31]:
class Car:
    def __init__(self, env):
        self.env = env
        # Start the run process everytime an instance is created.
        self.action = env.process(self.park_and_drive())

    def park_and_drive(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            # We yield the process that process() returns
            # to wait for it to finish
            yield self.env.process(self.charge(charge_duration))

            # The charge process has finished and
            # we can start driving again.
            print('Start driving at %d' % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)

    def charge(self, duration):
        yield self.env.timeout(duration)

The run process is automatically started when Car is instantiated. A new charge process is started every time the vehicle starts parking. By yielding the Process instance that Environment.process() returns, the run process starts waiting for it to finish.

Starting the simulation is straightforward again: We create an environment, one (or more) cars and finally call run().

In [32]:
env2 = simpy.Environment()
car = Car(env2)
#car2 = Car(env2)
env2.run(until=33)

Start parking and charging at 0
Start driving at 5
Start parking and charging at 7
Start driving at 12
Start parking and charging at 14
Start driving at 19
Start parking and charging at 21
Start driving at 26
Start parking and charging at 28


__Control questions (now it's your turn):__
* Alter the charging duration to 8 timesteps.
* Flip the order of the charging and the driving, so that the car first drives and thereafter charges.
* Implement the driving activity as a third process method.

## Shared Resources

SimPy offers three types of resources that help you modeling problems, where multiple processes want to use a resource of limited capacity (e.g., cars at a fuel station with a limited number of fuel pumps) or classical producer-consumer problems.

**Basic Resource Usage**

We’ll slightly modify our electric vehicle process car that we introduced in the last sections.

The car will now drive to a battery charging station (BCS) and request one of its two charging spots. If both of these spots are currently in use, it waits until one of them becomes available again. It then starts charging its battery and leaves the station afterwards:

In [34]:
class Car:
    def __init__(self, env, name):
        self.env = env
        self.name = name
        
    def drive(self, bcs, driving_time, charge_duration):
        
        # Simulate driving to the BCS
        yield self.env.timeout(driving_time)

          # Request one of its charging spots
        print('%s arriving at the charging station at %d' % (self.name, self.env.now))
        
        with bcs.request() as req:
                yield req #wait until a spot is available

                # Charge the battery
                print('%s starting to charge at %s' % (self.name, self.env.now))

                yield self.env.timeout(charge_duration)
                print('%s leaving the bcs at %s' % (self.name, self.env.now))

The resource’s request() method generates an event that lets you wait until the resource becomes available again. If you are resumed, you “own” the resource until you release it. 

A resource needs a reference to an Environment and a capacity when it is created:

In [49]:
env3 = simpy.Environment()
bcs = simpy.Resource(env3, capacity=2)

In [52]:
for i in range(10):
    car = Car(env=env3, name='Car %d' % i)
    env3.process(car.drive(bcs = bcs, driving_time = i*2, charge_duration =  5))

Finally, we can start the simulation. Since the car processes all terminate on their own in this simulation, we don’t need to specify an until time—the simulation will automatically stop when there are no more events left:

In [53]:
env3.run()

Car 0 arriving at the charging station at 27
Car 0 starting to charge at 27
Car 1 arriving at the charging station at 29
Car 1 starting to charge at 29
Car 2 arriving at the charging station at 31
Car 0 leaving the bcs at 32
Car 2 starting to charge at 32
Car 3 arriving at the charging station at 33
Car 1 leaving the bcs at 34
Car 3 starting to charge at 34
Car 4 arriving at the charging station at 35
Car 5 arriving at the charging station at 37
Car 2 leaving the bcs at 37
Car 4 starting to charge at 37
Car 6 arriving at the charging station at 39
Car 3 leaving the bcs at 39
Car 5 starting to charge at 39
Car 7 arriving at the charging station at 41
Car 4 leaving the bcs at 42
Car 6 starting to charge at 42
Car 8 arriving at the charging station at 43
Car 5 leaving the bcs at 44
Car 7 starting to charge at 44
Car 9 arriving at the charging station at 45
Car 6 leaving the bcs at 47
Car 8 starting to charge at 47
Car 7 leaving the bcs at 49
Car 9 starting to charge at 49
Car 8 leaving th

If you use the resource with the with statement as shown above, the resource is automatically being released. If you call request() without `with`, you are responsible to call release() once you are done using the resource.

When you release a resource, the next waiting process is resumed and now “owns” one of the resource’s slots. The basic Resource sorts waiting processes in a FIFO (first in—first out) way.

__Control questions (now it's your turn):__
* Set the capacity of the resource to 3 and re-run the simulation.
* Create more cars so that a queue develops at the charging station. 
* Try to model the resource in the car process without using the `with` statement. Don't forget to release the resource.

<img src="images/d3.png" style="width:50%; float:center;" />
